In [ ]:
# Add project src to path.
import set_path
import glob

# Import modules.
import numpy as np
import time
import logging
from tqdm import tqdm

# Visulatisation
#%matplotlib widget
import matplotlib.pyplot as plt

import set_path
import src.utils.ahn_utils as ahn_utils
import src.utils.las_utils as las_utils
import src.utils.plot_utils as plot_utils
import src.utils.clip_utils as clip_utils
import src.utils.math_utils as math_utils
import src.utils.bgt_utils as bgt_utils

from src.utils.interpolation import FastGridInterpolator

from pyntcloud import PyntCloud
import pandas as pd
from skimage import feature
from scipy import ndimage
import cv2 as cv
import pyransac3d as pyrsc

import open3d as o3d
from shapely.geometry import LineString, Point
from shapely.ops import nearest_points
import shapely 
from scipy.optimize import curve_fit
import scipy.spatial

from multiprocessing import Pool
from sklearn.cluster import DBSCAN

from skimage.transform import probabilistic_hough_line, hough_line, hough_line_peaks
from src.region_growing.label_connected_comp import LabelConnectedComp

from sklearn.metrics import classification_report

## Methods

In [ ]:
def plot_cloud(points, labels, plot_3d=True, legend=True, aspect=5):

    fig = plt.figure()
    if plot_3d:
        ax = plt.axes(projection='3d')
        for label in np.unique(labels):
            mask = labels == label
            ax.scatter(points[mask][:,0],points[mask][:,1],points[mask][:,2], s=2, label=str(label))
        ax.set_zlim((0,12))
        if aspect:
            centers = points.min(axis=0) + ((points.max(axis=0) - points.min(axis=0)) / 2)
            ax.set_xlim((centers[0]-aspect,centers[0]+aspect))
            ax.set_ylim((centers[1]-aspect,centers[1]+aspect))

    else:
        ax = plt.axes()
        for label in np.unique(labels):
            mask = labels == label
            ax.scatter(points[mask][:,0],points[mask][:,1], s=1,label=str(label))
    if legend:
        plt.legend()
    plt.show()

In [ ]:
def sample_cable(points, true_labels, sample_size=0.8):
    if sample_size == 1:
        return true_labels == 15
    else:
        cable_ids = np.where(true_labels==15)[0]
        cable_sample_ids = np.random.choice(cable_ids, int(len(cable_ids)*(1-sample_size)), replace=False)
        sample_labels = true_labels.copy()
        sample_labels[cable_sample_ids] = 0
        cable_mask = sample_labels == 15

        return cable_mask

In [ ]:
def main_direction(points):
    """ Returns the eigenvector corresponding to the largest eigenvalue of `points`"""
    cov = np.cov(points, rowvar=False)
    eig_val, eig_vec = np.linalg.eig(cov)
    dir_v = eig_vec[:,eig_val.argmax()]
    if dir_v[0] < 0:
        dir_v *= -1
    return dir_v

def cable_cut(points, mask):
    """Create a new axis along the direction of a cable. Cable start is 0"""
    cable_dir = main_direction(points[mask][:,:2])
    cable_dir_axis = np.dot(points[:,:2], cable_dir)
    cable_dir_axis -= cable_dir_axis[mask].min()
    return cable_dir_axis

def cable_to_linestrings(points, cable_axis):

    # start and end points
    start_idx = cable_axis.argmin()
    start_pt = [points[start_idx,0], points[start_idx,1]]
    end_idx = cable_axis.argmax()
    end_pt = [points[end_idx,0], points[end_idx,1]]
    
    # cable height LineString (along direction)
    point_spacing = 0.5
    mean_z, _, _ = scipy.stats.binned_statistic(cable_axis, points[:, 2], statistic='mean', bins=np.arange(0, cable_axis.max(), point_spacing))
    xs = np.arange((point_spacing/2), cable_axis.max() - (point_spacing/2), point_spacing)
    line_pts = np.vstack((xs, mean_z)).T
    line_pts = line_pts[~np.isnan(line_pts).any(axis=1)]
    cable_ylinestring = shapely.geometry.LineString(line_pts)

    # cable planar LineString
    point_spacing = 1
    mean_x, _, _ = scipy.stats.binned_statistic(cable_axis, points[:, 0], statistic='mean', bins=np.arange(0, cable_axis.max(), point_spacing))
    mean_y, _, _ = scipy.stats.binned_statistic(cable_axis, points[:, 1], statistic='mean', bins=np.arange(0, cable_axis.max(), point_spacing))
    line_pts = np.vstack((mean_x, mean_y)).T
    line_pts = np.vstack((line_pts, start_pt, end_pt))
    line_pts = line_pts[~np.isnan(line_pts).any(axis=1)]
    cable_axislinestring = shapely.geometry.LineString(line_pts)

    return cable_ylinestring, cable_axislinestring

def clip_cable_area(points, mask, cable_axis, h_buffer=0.5, w_buffer=.5):

    # Cable directional and height line
    cable_yline, cable_axisline = cable_to_linestrings(points[mask], cable_axis[mask])

    # Height clip
    cable_ypoly = cable_yline.buffer(h_buffer)
    dir_points = np.vstack((cable_axis, points[:,2])).T
    height_clip_mask = clip_utils.poly_clip(dir_points, cable_ypoly)

    # Direction clip
    cable_axispoly = cable_axisline.buffer(w_buffer)
    axis_clip_mask = clip_utils.poly_clip(points[:,:2], cable_axispoly)

    # Clip
    clip_mask = np.logical_and(height_clip_mask, axis_clip_mask)

    return clip_mask, cable_yline, cable_axisline


In [ ]:
np.array([[1,2],[1,1]])

In [ ]:
def _has_decoration(points, cable_mask, cable_axis, cable_axisline, cable_yline, plot=False, threshold=.5, width_buffer=.2):

    # create cable bottom polygon
    search_height = 1
    cable_radius = 0.1
    top_coords = np.array(cable_yline.coords) - [0, cable_radius]
    bottom_coords = np.array(cable_yline.coords) - [0, cable_radius + search_height]
    clip_poly = shapely.geometry.Polygon([*top_coords, *bottom_coords[::-1]])
    dir_points = np.vstack((cable_axis, points[:,2])).T
    
    # decoration clip
    mask = np.all([
        ~cable_mask,
        clip_utils.poly_clip(points[:,:2], cable_axisline.buffer(width_buffer)), # optional to make have smaller width buffer
        clip_utils.poly_clip(dir_points, clip_poly), # clip points under the line
        cable_axis > .25, 
        cable_axis < cable_axis[cable_mask].max() - .25
        ], axis=0)

    # density analysis
    res = scipy.stats.binned_statistic(cable_axis[mask], np.ones(mask.sum()), statistic='count', bins=np.arange(0.25, cable_axis[cable_mask].max()-.25, 0.2)) # TODO: right margins
    pt_threshold = 10
    bin_density = res.statistic > pt_threshold
    cable_density_per = bin_density.sum() / len(bin_density)
    result = cable_density_per > threshold

    if plot:
        ax1 = plt.subplot()
        ax1.set_title('Decoration Analysis')
        ax1.scatter(cable_axis[mask], points[mask][:,2], s=1, c='blue')
        ax1.scatter(cable_axis[cable_mask], points[cable_mask][:,2], s=1, c='gray')
        bin_xs = (res.bin_edges[1:] + res.bin_edges[:-1]) / 2
        ax2 = ax1.twinx()
        ax2.plot(bin_xs, res.statistic, c='k')
        

    return result

In [ ]:
def projection_filter(points, cable_axis, plot=True, num_dil=3):

    # Project
    grid, _, _, binnumber = scipy.stats.binned_statistic_2d(cable_axis, points[:,2], np.ones(len(points)), statistic='count', bins=[np.arange(cable_axis.min(), cable_axis.max(), 0.05), np.arange(points[:,2].min(), points[:,2].max(), 0.05)], expand_binnumbers=True)
    grid_shape = np.array(grid.shape) + 1
    binindices = np.ravel_multi_index(binnumber-1, grid_shape)

    # Image filters
    grid = np.where(grid>0, True, False)
    grid_1 = ndimage.binary_fill_holes(grid)
    grid_2 = ndimage.binary_erosion(grid_1, iterations=2)
    grid_3 = ndimage.binary_dilation(grid_2, iterations=num_dil)

    if plot:
        fig, ax = plt.subplots(4,1, sharex=True)
        ax[0].set_title('Projection Filters')
        ax[0].imshow(grid.T, cmap = 'gray')
        ax[1].imshow(grid_1.T, cmap = 'gray')
        ax[2].imshow(grid_2.T, cmap = 'gray')
        ax[3].imshow(grid_3.T, cmap = 'gray')
        ax[0].invert_yaxis()
        ax[1].invert_yaxis()
        ax[2].invert_yaxis()
        ax[3].invert_yaxis()

    # convert 2d back to 3d mask
    multi_index = np.array(np.where(grid_3>0))
    residual_bins = np.ravel_multi_index(multi_index, grid_shape)
    residual_mask = np.isin(binindices, residual_bins)

    return residual_mask

def segmentation_plot(points, cable_axis, segmentation, filter_mask, cable_yline):

    # Plot clusters
    fig = plt.figure()
    ax = plt.axes()
    for cluster in np.unique(segmentation.labels_):
        c_mask = segmentation.labels_ == cluster
        ax.scatter(cable_axis[filter_mask][c_mask], points[filter_mask][c_mask][:,2], s=1)
        ax.scatter(cable_axis[filter_mask][c_mask].mean(), points[filter_mask][c_mask][:,2].mean(), c='k')
    #ax.scatter(cable_axis[filter_mask], points[filter_mask][:,2], s=1)
    ax.scatter(cable_axis[~filter_mask], points[~filter_mask][:,2], s=1, c='gray')

    yline = np.array(cable_yline.coords)
    ax.plot(yline[:,0], yline[:,1], c='k')

    ax.set_ylim((points[:,2].min()-1,points[:,2].max()+1))

def _is_armatuur(points, cable_axis, cable_yline, cable_axisline):

    # Cluster dimension
    cl_width = cable_axis.max() - cable_axis.min()
    cl_height = points[:,2].max() - points[:,2].min()

    # Cluster position offset
    cl_centroid = points.mean(axis=0)
    axis_coord = cable_axis.mean()
    y_offset = nearest_points(cable_yline, Point([axis_coord, cl_centroid[2]]))[0].y - cl_centroid[2]
    axis_offset = LineString(nearest_points(cable_axisline, Point(cl_centroid[:2]))).length

    # Check conditions (1. cluster dimensions & 2. centroid should be under cable)
    width_cond = .25 < cl_width < 1.2
    height_cond = .25 < cl_height < .6
    yoff_cond = 0 < y_offset < .55
    xyoff_cond = axis_offset < 0.1
    valid_cluster = np.all([width_cond, height_cond, yoff_cond, xyoff_cond])

    record = [axis_coord, cl_width, cl_height, y_offset, axis_offset, valid_cluster]

    return valid_cluster, record




## Approach: PRJ

In [ ]:
def armatuur_extractor_prj(points, cable_mask, true_labels, plot=False):

    # 0. create mask
    armatuur_mask = np.zeros(len(points), dtype=bool)
    label_mask = np.zeros(len(points))
    label_mask[true_labels==15] = 15

    # 1. Clip search-area
    cable_axis = cable_cut(points, cable_mask)
    clip_mask, cable_yline, cable_axisline = clip_cable_area(points, cable_mask, cable_axis)
    
    # 3. Check for decoration
    decoration_cable = _has_decoration(points[clip_mask], cable_mask[clip_mask], cable_axis[clip_mask], cable_axisline, cable_yline, plot)
    if not decoration_cable:

        # 4. Projection
        filter_mask = projection_filter(points[clip_mask], cable_axis[clip_mask], plot, 5)

        # 5. ConnectedComponents
        filter_mask = ~cable_mask[np.where(clip_mask)[0]] & filter_mask
        lcc = LabelConnectedComp(grid_size=0.1, min_component_size=40)
        point_components = lcc.get_components(points[clip_mask][filter_mask])
        cc_labels = np.unique(point_components)

        if plot:
            
            fig, ax = plt.subplots(1,len(set(cc_labels).difference((-1,))), figsize=(12,4))
            i=0
            for cc in set(cc_labels).difference((-1,)):
                cc_mask = (point_components == cc)
                cc_pts = points[clip_mask][filter_mask][cc_mask][:,2]
                cc_axis_pts = cable_axis[clip_mask][filter_mask][cc_mask]

                ax[i].scatter(cc_axis_pts, cc_pts, s=1)
                ax[i].scatter(cable_axis[cable_mask], points[cable_mask][:,2], s=1, c='k')
                m_ax = cc_axis_pts.min() + (cc_axis_pts.max()-cc_axis_pts.min())/2
                m_y = cc_pts.min() + (cc_pts.max()-cc_pts.min())/2
                

                bbox = math_utils.compute_bounding_box(np.vstack([cc_axis_pts,cc_pts]).T)
                dims = [round(bbox[2]-bbox[0],2), round(bbox[3]-bbox[1],2)]

                ploy = shapely.geometry.box(*bbox)
                ax[i].plot(*ploy.exterior.xy)
                margin = np.max(dims)/2 + 0.2
                ax[i].set_xlim((m_ax-margin, m_ax+margin))
                ax[i].set_ylim((m_y-margin, m_y+margin))
                ax[i].set_title(dims)
                i+=1

        # 6. Cluster analysis for armatuur
        cc_count = 0
        for cc in set(cc_labels).difference((-1,)):
            # select points that belong to the cluster
            cc_mask = (point_components == cc)
            cc_pts = points[clip_mask][filter_mask][cc_mask]
            cc_axis_pts = cable_axis[clip_mask][filter_mask][cc_mask]
            valid, record = _is_armatuur(cc_pts, cc_axis_pts, cable_yline, cable_axisline)
            if plot:
                print(cc, valid, record)
            if valid:
                armatuur_mask[np.where(clip_mask)[0][filter_mask][cc_mask]] = valid

    if plot:
        plot_cloud(points[clip_mask], armatuur_mask[clip_mask])

    return armatuur_mask, clip_mask


In [ ]:
results = []

#for las_file in ['../datasets/suspended_light_cables/sample8.laz']:
for las_file in glob.iglob('../datasets/suspended_light_cables/*.laz'):

    pointcloud = las_utils.read_las(las_file)
    true_labels = pointcloud.label
    true_labels[~((true_labels == 15) | (true_labels == 14))] = 0
    points = np.vstack((pointcloud.x, pointcloud.y, pointcloud.z)).T
    labels = np.zeros(len(points))

    # Sample cable mask?
    cable_mask = sample_cable(points, true_labels, 1)

    # Armatuur extractor
    armatuur_mask, clip_mask = armatuur_extractor_prj(points, cable_mask, true_labels, False)

    label_mask = np.zeros(len(points))
    label_mask[true_labels==15] = 15
    label_mask[armatuur_mask] = 14
    report = classification_report(true_labels[clip_mask], label_mask[clip_mask], labels=[0,14,15], target_names=['unclassified', 'armatuur', 'cable'], output_dict=True, zero_division=1)['armatuur']
    print('Processed file', las_file[-12:-4], '\tf1:',round(report['f1-score'],2))
    results.append(report)

df = pd.DataFrame(results)[['precision','recall','f1-score']]
df.boxplot()
df.mean()

## Approach: LCC

In [ ]:
def armatuur_extractor_cc(points, cable_mask, plot=False):

    # 0. create mask
    armatuur_mask = np.zeros(len(points), dtype=bool)

    # 1. Clip search-area
    cable_axis = cable_cut(points, cable_mask)
    clip_mask, cable_yline, cable_axisline = clip_cable_area(points, cable_mask, cable_axis, 1, .5)

    # 2. Check for decoration
    decoration_cable = _has_decoration(points[clip_mask], cable_mask[clip_mask], cable_axis[clip_mask], cable_axisline, cable_yline)
    if not decoration_cable:

        # 3. ConnectedComponents
        mask = clip_mask & ~cable_mask
        lcc = LabelConnectedComp(label=0, grid_size=0.1,
                                min_component_size=20)
        point_components = lcc.get_components(points[mask])
        cc_labels = np.unique(point_components)

        # 4. Cluster analysis for armatuur
        cc_count = 0
        for cc in set(cc_labels).difference((-1,)):
            # select points that belong to the cluster
            cc_mask = (point_components == cc)

            cc_pts = points[mask][cc_mask]
            cc_axis_pts = cable_axis[mask][cc_mask]

            valid, _ = _is_armatuur(cc_pts, cc_axis_pts, cable_yline, cable_axisline)
            if valid:
                armatuur_mask[np.where(mask)[0][cc_mask]] = True
                cc_count += 1

    if plot:
        # plot_cloud(points[mask], point_components)
        plot_cloud(points[mask], armatuur_mask[mask])
        
    return armatuur_mask, clip_mask

In [ ]:
results = []

#for las_file in ['../datasets/suspended_light_cables/sample7.laz']:
for las_file in glob.iglob('../datasets/suspended_light_cables/*.laz'):

    pointcloud = las_utils.read_las(las_file)
    true_labels = pointcloud.label
    true_labels[~((true_labels == 15) | (true_labels == 14))] = 0
    points = np.vstack((pointcloud.x, pointcloud.y, pointcloud.z)).T
    labels = np.zeros(len(points))

    # sample cable mask
    cable_mask = sample_cable(points, true_labels, 1)

    # armatuur_extractor_cc
    armatuur_mask, clip_mask = armatuur_extractor_cc(points, cable_mask, False)

    label_mask = np.zeros(len(points))
    label_mask[true_labels==15] = 15
    label_mask[armatuur_mask] = 14
    report = classification_report(true_labels[clip_mask], label_mask[clip_mask], labels=[0,14,15], target_names=['unclassified', 'armatuur', 'cable'], output_dict=True, zero_division=1)['armatuur']
    #print('Processing file', las_file, '\tf1:',round(report['f1-score'],2))
    results.append(report)

df = pd.DataFrame(results)[['precision','recall','f1-score']]
df.boxplot()
df.mean()

## Approach: 
Cable-bending

In [ ]:
def unit_vector(v1):
    """ Returns the unit vector of `v1`"""
    return v1 / np.linalg.norm(v1)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'"""
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.rad2deg(np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0)))

In [ ]:
def cable_angle(points, labels, plot=True):

    cable_mask = labels == 15
    lamp_mask = labels == 14

    # Calculate directional axis
    cable_dir_axis = cable_cut(points, cable_mask)

    # cable height LineString (along direction)
    mean_z, _, _ = scipy.stats.binned_statistic(cable_dir_axis[cable_mask], points[cable_mask][:, 2], statistic='mean', bins=np.arange(0, cable_dir_axis.max(), 0.5))
    xs = np.arange(0.25, cable_dir_axis.max()-0.25, 0.5)
    ls_poitns = np.vstack((xs, mean_z)).T
    cable_height_linestring = shapely.geometry.LineString(ls_poitns)

    # Calculate angle change
    color_v = []
    for i in range(len(ls_poitns)):
        if i < 5 or i > len(ls_poitns) - 6:
            color_v.append(0.)
        else:
            v_a = ls_poitns[i] - ls_poitns[i-5]
            v_b = ls_poitns[i+5] - ls_poitns[i]
            angle = angle_between(v_a, v_b)
            color_v.append(angle)

    if plot:
        fig = plt.figure()
        ax = plt.axes()
        ax.scatter(xs, mean_z, s=4, c=color_v, cmap='Reds_r')
        ax.scatter(cable_dir_axis[lamp_mask], points[lamp_mask,2], s=2)

In [ ]:
cable_angle(points, true_labels)

In [ ]:
for las_file in glob.iglob('../datasets/suspended_light_cables/*.laz'):
    pointcloud = las_utils.read_las(las_file)
    true_labels = pointcloud.label
    points = np.vstack((pointcloud.x, pointcloud.y, pointcloud.z)).T
    labels = np.zeros(len(points))

    print('Processing file', las_file)
    cable_angle(points, true_labels)